In [1]:
import pandas as pd
import networkx as nx
import networkit as nk
import json

In [2]:
df = pd.read_csv('../dataset/subsample.csv')
G = nx.read_gml('../network/reviews_network.gml')

In [11]:
df['adamic_adar'].sort_values(by='').head()

,rating,parent_asin,user_id,main_category,average_rating,rating_number,price,neg,neu,pos,compound,user_deg,product_deg,user_pagerank,product_pagerank,adamic_adar
0,5,B0887P21Z2,AHGAOIZVODNHYMNCBV4DECZH42UQ,computers,4.8,48733.0,76.48,0.08,0.764,0.156,0.4738,0.000295,0.004321,0.000017,0.000185,0
1,5,B0BPN2DXC1,AHGAOIZVODNHYMNCBV4DECZH42UQ,computers,4.6,20671.0,54.98,0.00,1.000,0.000,0.0000,0.000295,0.003767,0.000017,0.000162,0
2,1,B09JKYBVJD,AHGAOIZVODNHYMNCBV4DECZH42UQ,computers,4.3,6168.0,85.99,0.00,0.859,0.141,0.6808,0.000295,0.003287,0.000017,0.000144,0
3,5,B008GVOVK0,AHGAOIZVODNHYMNCBV4DECZH42UQ,home audio & theater,4.0,14281.0,10.49,0.00,0.891,0.109,0.8061,0.000295,0.008938,0.000017,0.000378,0
4,5,B00TOVTZ7K,AHGAOIZVODNHYMNCBV4DECZH42UQ,home audio & theater,4.5,26636.0,18.79,0.00,0.280,0.720,0.8834,0.000295,0.004838,0.000017,0.000205,0


In [4]:
len(df)

295218

In [5]:
nkG = nk.nxadapter.nx2nk(G)

In [6]:
deg = nk.centrality.DegreeCentrality(nkG, normalized=True)
deg.run()
degreeCentrality = dict(zip(G.nodes(), deg.scores()))

df['user_deg'] = df['user_id'].map(degreeCentrality)
df['product_deg'] = df['parent_asin'].map(degreeCentrality)

In [7]:
pageRank = nx.pagerank(G)
df['user_pagerank'] = df['user_id'].map(pageRank)
df['product_pagerank'] = df['parent_asin'].map(pageRank)

In [10]:
adamicAdar_scores = {}

ebunch = list(zip(df['user_id'], df['parent_asin']))

for u, v, score in nx.adamic_adar_index(G, ebunch=ebunch):
    adamicAdar_scores[(u, v)] = score

def get_adamic_adar(row):
    return adamicAdar_scores.get((row['user_id'], row['parent_asin']), 0.0)

df['adamic_adar'] = df.apply(get_adamic_adar, axis=1)

In [11]:
eigCentrality = nx.eigenvector_centrality(G)
df['user_eig'] = df['user_id'].map(eigCentrality)
df['product_eig'] = df['parent_asin'].map(eigCentrality)

PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')

In [ ]:
cloCentrality = nx.closeness_centrality(G)
df['user_clo'] = df['user_id'].map(cloCentrality)
df['product_clo'] = df['parent_asin'].map(cloCentrality)

In [9]:
df.to_csv('../dataset/new_features.csv', index=False)